In [1]:
# ----------------------------
# Imports
# ----------------------------
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.isotonic import IsotonicRegression
from scipy.special import softmax
import numpy as np
import pandas as pd
import os

# ----------------------------
# Device
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ----------------------------
# Paths & Hyperparameters
# ----------------------------
data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/train"
test_data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/test"
num_classes = 9
batch_size = 32
lr = 1e-4
epochs = 5
k_folds = 5
test_split_ratio = 0.05

# ----------------------------
# Transforms - use official ImageNet stats!
# ----------------------------

mean = [0.3789, 0.3898, 0.3800]
std = [0.2241, 0.2245, 0.2227]

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# ----------------------------
# Load Datasets
# ----------------------------
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_tfms)
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=val_tfms)
num_samples = len(full_dataset)

# ----------------------------
# Initial Split for Hold-out Test
# ----------------------------
test_size = int(test_split_ratio * num_samples)
trainval_size = num_samples - test_size

indices = np.arange(num_samples)
np.random.shuffle(indices)
trainval_indices = indices[:trainval_size]
holdout_test_indices = indices[trainval_size:]

trainval_dataset = Subset(full_dataset, trainval_indices)
holdout_test_dataset = Subset(full_dataset, holdout_test_indices)
holdout_test_dataset.dataset.transform = val_tfms

# For stratified k-fold: get labels for trainval split
trainval_labels = [full_dataset.targets[i] for i in trainval_indices]

# ----------------------------
# Stratified K-Fold CV
# ----------------------------
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
fold_best_metrics = []
fold_best_models = []
for fold, (fold_train_idx, fold_val_idx) in enumerate(skf.split(trainval_indices, trainval_labels)):
    print(f"\n========== Fold {fold+1}/{k_folds} ==========")
    train_indices = [trainval_indices[i] for i in fold_train_idx]
    val_indices = [trainval_indices[i] for i in fold_val_idx]
    train_subset = Subset(full_dataset, train_indices)
    val_subset = Subset(full_dataset, val_indices)
    val_subset.dataset.transform = val_tfms

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2)

    model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
    for param in model.parameters():
        param.requires_grad = True   # <-- FULL FINE-TUNING
    in_features = model.classifier[2].in_features
    model.classifier[2] = nn.Linear(in_features, num_classes)
    model = model.to(device)


    fold_train_labels = [full_dataset.targets[i] for i in train_indices]
    class_counts = np.bincount(fold_train_labels, minlength=num_classes)
    class_weights = 1.0 / (np.sqrt(np.maximum(class_counts, 1)))
    weights_tensor = torch.FloatTensor(class_weights).to(device)

    criterion = nn.CrossEntropyLoss(weight=weights_tensor)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_metric = -1
    best_model_state = None
    best_filename = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1} [Train]"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}] Loss: {running_loss/len(train_loader):.4f}")

        model.eval()
        all_labels, all_preds = [], []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Fold {fold+1} Epoch {epoch+1} [Val]"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())
        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)

        cm = confusion_matrix(all_labels, all_preds, labels=range(num_classes))
        per_class_acc = cm.diagonal() / cm.sum(axis=1)
        per_class_prec = precision_score(all_labels, all_preds, labels=range(num_classes), average=None, zero_division=0)
        per_class_rec = recall_score(all_labels, all_preds, labels=range(num_classes), average=None, zero_division=0)
        overall_acc = np.mean(all_preds == all_labels) * 100
        overall_prec = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        overall_rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)

        print(f"\n{'='*15} Fold {fold+1} Epoch {epoch+1} Validation Metrics {'='*15}")
        print(f"{'Class':<8}{'Accuracy':>10}{'Precision':>12}{'Recall':>10}")
        print("-" * 40)
        for i in range(num_classes):
            acc = per_class_acc[i] if not np.isnan(per_class_acc[i]) else 0.0
            prec = per_class_prec[i] if not np.isnan(per_class_prec[i]) else 0.0
            rec = per_class_rec[i] if not np.isnan(per_class_rec[i]) else 0.0
            print(f"{i:<8}{acc:>10.4f}{prec:>12.4f}{rec:>10.4f}")
        print("-" * 40)
        print(f"{'Overall':<8}{overall_acc:>10.2f}{overall_prec*100:>12.2f}{overall_rec*100:>10.2f}")
        print(f"{'='*55}\n")

        composite_metric = overall_acc + overall_prec * 100 + overall_rec * 100
        if composite_metric > best_metric:
            best_metric = composite_metric
            best_model_state = model.state_dict()
            best_filename = f"bestmodel_fold{fold+1}_epoch{epoch+1}_acc{overall_acc:.2f}_prec{overall_prec:.4f}_rec{overall_rec:.4f}.pt"
            torch.save(best_model_state, best_filename)
            print(f"Best model for fold {fold+1} updated and saved as {best_filename}")

    fold_best_metrics.append(best_metric)
    fold_best_models.append(best_filename)

# ----------------------------
# Retrain Best Fold Model On Full Trainval
# ----------------------------
best_fold_idx = np.argmax(fold_best_metrics)
best_model_path = fold_best_models[best_fold_idx]
print(f"\nUsing best model from fold {best_fold_idx+1}: {best_model_path}")

model = convnext_tiny(weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
for param in model.parameters():
    param.requires_grad = True   # <-- FULL FINE-TUNING
in_features = model.classifier[2].in_features
model.classifier[2] = nn.Linear(in_features, num_classes)
model = model.to(device)
model.load_state_dict(torch.load(best_model_path))

train_labels = [full_dataset.targets[i] for i in trainval_indices]
class_counts = np.bincount(train_labels, minlength=num_classes)
class_weights = 1. / (np.sqrt(np.maximum(class_counts, 1)))
weights_tensor = torch.FloatTensor(class_weights).to(device)
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

full_train_loader = DataLoader(trainval_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(full_train_loader, desc=f"Retrain Epoch {epoch+1}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}] Loss: {running_loss/len(full_train_loader):.4f}")

# ----------------------------
# CALIBRATION AND TEST EVALUATION
# ----------------------------
# --- Collect validation logits/labels for calibration
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2)
val_logits, val_labels = [], []
model.eval()
with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Collecting validation logits"):
        images = images.to(device)
        outputs = model(images)
        val_logits.append(outputs.cpu().numpy())
        val_labels.append(labels.cpu().numpy())
val_logits = np.concatenate(val_logits)
val_labels = np.concatenate(val_labels)
val_probs = softmax(val_logits, axis=1)

# --- Fit isotonic regression per class
calibrators = []
for i in range(num_classes):
    y_bin = (val_labels == i).astype(int)
    calibrator = IsotonicRegression(out_of_bounds="clip")
    calibrator.fit(val_probs[:, i], y_bin)
    calibrators.append(calibrator)

# --- Gather logits for hold-out test set
holdout_test_loader = DataLoader(holdout_test_dataset, batch_size=1, shuffle=False, num_workers=2)
test_logits, test_labels = [], []
with torch.no_grad():
    for images, labels in tqdm(holdout_test_loader, desc="Holdout test logits"):
        images = images.to(device)
        outputs = model(images)
        test_logits.append(outputs.cpu().numpy())
        test_labels.append(labels.cpu().numpy())
test_logits = np.concatenate(test_logits)
test_labels = np.concatenate(test_labels)
test_probs = softmax(test_logits, axis=1)

# --- Calibrate hold-out test probabilities
cal_test_probs = np.zeros_like(test_probs)
for i in range(num_classes):
    cal_test_probs[:, i] = calibrators[i].transform(test_probs[:, i])
cal_test_preds = np.argmax(cal_test_probs, axis=1)

# --- Print calibrated test metrics
cm = confusion_matrix(test_labels, cal_test_preds, labels=range(num_classes))
per_class_acc = cm.diagonal() / cm.sum(axis=1)
per_class_prec = precision_score(test_labels, cal_test_preds, labels=range(num_classes), average=None, zero_division=0)
per_class_rec = recall_score(test_labels, cal_test_preds, labels=range(num_classes), average=None, zero_division=0)
overall_acc = accuracy_score(test_labels, cal_test_preds) * 100
overall_prec = precision_score(test_labels, cal_test_preds, average='macro', zero_division=0)
overall_rec = recall_score(test_labels, cal_test_preds, average='macro', zero_division=0)

print(f"\n{'='*15} Hold-out Test Metrics (Calibrated) {'='*15}")
print(f"{'Class':<8}{'Accuracy':>10}{'Precision':>12}{'Recall':>10}")
print("-" * 40)
for i in range(num_classes):
    acc = per_class_acc[i] if not np.isnan(per_class_acc[i]) else 0.0
    prec = per_class_prec[i] if not np.isnan(per_class_prec[i]) else 0.0
    rec = per_class_rec[i] if not np.isnan(per_class_rec[i]) else 0.0
    print(f"{i:<8}{acc:>10.4f}{prec:>12.4f}{rec:>10.4f}")
print("-" * 40)
print(f"{'Overall':<8}{overall_acc:>10.2f}{overall_prec*100:>12.2f}{overall_rec*100:>10.2f}")
print(f"{'='*55}\n")


Using device: cuda

========== Fold 1/5 ==========


Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:01<00:00, 78.6MB/s] 
Fold 1 Epoch 1 [Train]: 100%|██████████| 292/292 [02:40<00:00,  1.82it/s]


Epoch [1] Loss: 0.6371


Fold 1 Epoch 1 [Val]: 100%|██████████| 73/73 [00:11<00:00,  6.16it/s]



=============== Fold 1 Epoch 1 Validation Metrics ===============
Class     Accuracy   Precision    Recall
----------------------------------------
0           0.6887      0.9369    0.6887
1           0.8741      0.9843    0.8741
2           0.9373      0.9660    0.9373
3           1.0000      0.9172    1.0000
4           0.9924      0.9091    0.9924
5           0.9790      0.8917    0.9790
6           0.9699      0.9214    0.9699
7           0.9728      0.8034    0.9728
8           0.9044      0.8092    0.9044
----------------------------------------
Overall      92.87       90.44     92.43

Best model for fold 1 updated and saved as bestmodel_fold1_epoch1_acc92.87_prec0.9044_rec0.9243.pt


...

Epoch [4] Loss: 0.0368


Retrain Epoch 5: 100%|██████████| 364/364 [03:35<00:00,  1.69it/s]


Epoch [5] Loss: 0.0192


Holdout test logits: 100%|██████████| 612/612 [00:05<00:00, 112.57it/s]


=============== Hold-out Test Metrics (Calibrated) ===============
Class     Accuracy   Precision    Recall
----------------------------------------
0           0.9143      0.9143    0.9143
1           0.9737      0.9487    0.9737
2           0.9808      0.9503    0.9808
3           0.8750      0.9545    0.8750
4           0.9655      0.9655    0.9655
5           0.8919      0.9706    0.8919
6           0.9667      1.0000    0.9667
7           0.8974      0.9459    0.8974
8           0.9545      0.9767    0.9545
----------------------------------------
Overall      95.42       95.85     93.55



In [2]:
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from scipy.special import softmax

# Set model to evaluation mode
model.eval()

test_dataset = datasets.ImageFolder(root=test_data_dir, transform=val_tfms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

submission_rows = []
with torch.no_grad():
    for batch_idx, (images, _) in enumerate(tqdm(test_loader, desc="Generating Submission")):
        batch_start = batch_idx * test_loader.batch_size
        filenames = [
            test_dataset.imgs[batch_start + i][0].split('/')[-1]
            for i in range(len(images))
        ]
        images = images.to(device)
        outputs = model(images)                       # [batch, num_classes]
        logits = outputs.cpu().numpy()                # [batch, num_classes]
        probs = softmax(logits, axis=1)               # [batch, num_classes]

        # Calibrate probabilities using fitted calibrators
        calibrated_probs = np.zeros_like(probs)
        for i in range(num_classes):
            calibrated_probs[:, i] = calibrators[i].transform(probs[:, i])
        pred_labels = np.argmax(calibrated_probs, axis=1)

        for fname, label in zip(filenames, pred_labels):
            submission_rows.append({'filename': fname, 'label': label})

submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv('submission.csv', index=False)
print("Saved sample submission to submission.csv")


Generating Submission: 100%|██████████| 165/165 [00:27<00:00,  6.10it/s]

Saved sample submission to submission.csv
